# 逻辑回归
## 自己编写函数实现（准确率：0.7781）

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

iter_num = 50 #迭代次数

def readData():
    data = pd.read_csv('train.csv')
    Xcol=[col for col in data.columns if col != 'IsAlert']
    X = data[Xcol]
    Y = data['IsAlert']
    Xtrain, Xtest, ytrain, ytest= train_test_split(X, Y, test_size=0.20)

    #对数据进行简化处理
    Xcol=[col for col in data.columns if col != 'IsAlert']
    for i in Xcol:
        Xtrain[i] = (Xtrain[i] - Xtrain[i].mean())/(Xtrain[i].max() - Xtrain[i].min())

    for i in Xcol:
        Xtest[i] = (Xtest[i] - Xtest[i].mean())/(Xtest[i].max() - Xtest[i].min())
    
#     Xtrain.head()
    return Xtrain, Xtest, ytrain, ytest

    
# Logistic Regression
def sigmoid(inX):
    return 1.0/(1+np.exp(-inX))

# 随机梯度下降算法
def gradDescent(dataMatrix, classLabels):
    #实现随机梯度下降 theta=theta+alpha*(Y-h)*X    h=sigmoid(sum(theta*X))
    m,n=dataMatrix.shape
    print('数据大小：',m,n)
    weights=np.ones(n)
#     print('weights:',weights)
    alpha=0.01#步长
    for i in range(iter_num):#迭代次数
        for j in range(m):
            h=sigmoid(sum(dataMatrix[j]*weights))
            error=classLabels[j]-h
            weights=weights+alpha*error*dataMatrix[j]
#             print('h:',h,'error:',error)#,'weights:',weights
    return weights
            

# 分类决策函数
def classifyVector(inX,weights):
    #根据随机梯度下降训练的参数，对测试数据进行预测
#     pass
    prob=sigmoid(np.sum(inX*weights))
    if prob>0.5:
        return 1
    elif prob<0.5:
        return 0

def classifytest(dataMatrix, classLabels, weights):
    #基于classifyVector，统计预测的正确率
    m,n=dataMatrix.shape
    RightCounter=0
    WrongCounter=0
    for j in range(m):
        Result=classifyVector(dataMatrix[j],weights)
        if Result==int(classLabels[j]):
            RightCounter=RightCounter+1
        else:
            WrongCounter=WrongCounter+1
    print("Accuracy:",RightCounter/(RightCounter+WrongCounter))

Xtrain, Xtest, ytrain, ytest = readData()
weights = gradDescent(np.array(Xtrain), np.array(ytrain))
print('Weights:',weights)
classifytest( np.array(Xtest), np.array(ytest), weights)


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


数据大小： 483463 27
Weights: [-1.92253344e+00 -2.54309701e-01 -3.34394989e-01 -3.37154924e-01
  6.12964700e+00  7.62684379e+00  4.87589193e+00  1.12701624e+00
 -3.24726164e-01  3.21202256e-01 -4.45248544e-02 -1.33631198e+00
 -4.19034726e+00 -2.98558228e+00 -2.08261893e+00  3.79678884e+00
  8.03239947e-01 -3.59243565e-02 -1.70272412e+00 -6.32523240e-01
 -2.62351309e-01 -2.13251242e-01 -2.50905940e-01  1.86195381e+00
 -4.88718733e-01 -6.92847528e-01  4.90825667e+01]
Accuracy: 0.778060000330945


## Scikit-learn实现逻辑回归（准确率：0.7923）

* 导入库，读取数据

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
data=pd.read_csv('train.csv')
data.head()

,IsAlert,P1,P2,P3,P4,P5,P6,P7,E1,E2,...,E11,V1,V2,V3,V4,V5,V6,V7,V8,V9
0,0,34.7406,9.84593,1400,42.8571,0.290601,572,104.895,0.0,0.0,...,0.0,101.96,0.175,752,5.99375,0,2005,13.4,4,14.8004
1,0,34.4215,13.41120,1400,42.8571,0.290601,572,104.895,0.0,0.0,...,0.0,101.98,0.455,752,5.99375,0,2007,13.4,4,14.7729
2,0,34.3447,15.18520,1400,42.8571,0.290601,576,104.167,0.0,0.0,...,0.0,101.97,0.280,752,5.99375,0,2011,13.4,4,14.7736
3,0,34.3421,8.84696,1400,42.8571,0.290601,576,104.167,0.0,0.0,...,0.0,101.99,0.070,752,5.99375,0,2015,13.4,4,14.7667
4,0,34.3322,14.69940,1400,42.8571,0.290601,576,104.167,0.0,0.0,...,0.0,102.07,0.175,752,5.99375,0,2017,13.4,4,14.7757


* 分割数据，未对原始数据就行归一化处理，得到准确率为0.7873

In [3]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(data[[col for col in data.columns if col != 'IsAlert']], data['IsAlert'])
model=LogisticRegression(C=1,penalty='l2')
model.fit(Xtrain,Ytrain)
model.score(Xtest,Ytest)

0.7873420570150182

* 分割数据，对原始数据就行归一化处理，得到准确率为0.7923，准确率略微提高

In [4]:
Xcol=[col for col in data.columns if col != 'IsAlert']
for col in Xcol:
    data[col]=(data[col]-data[col].min())/(data[col].max()-data[col].min())

In [5]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(data[[col for col in data.columns if col != 'IsAlert']], data['IsAlert'])
model=LogisticRegression(C=1,penalty='l2')
model.fit(Xtrain,Ytrain)
model.score(Xtest,Ytest)

0.7923194535454022

## Scikit-learn实现随机森林算法（准确率：0.9943）

In [6]:
from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import GridSearchCV
model_r=RandomForestClassifier(n_estimators=100,n_jobs= -1)
model_r.fit(Xtrain,Ytrain)
model_r.score(Xtest,Ytest)

0.9943408589980341

# 结论
* 自己编写的函数运行时间极长，且准确率比 Scikit-learn的算法准确率还低；